Getting started with TRIOSlib
=======================

In [3]:
%%html
<style>
table,tr,td {border:none!important}
</style>

This document intends to give an overview of TRIOS and its main classes. If you are not familiar with $W$-operator learning, we suggest reading this [Introduction to Image Operator Learning](http://www.vision.ime.usp.br/projects/trios/basic_formulation/).

<table>
    <tr>
        <td class="noborder"> <img src="_static/image1.png"> </td> <td> <img src="_static/image1-out.png"> </td>
    <tr>
    <tr>
        <td style="text-align: center;"> (a) Example input image </td> <td style="text-align: center;"> (b) Example output image </td>
    </tr>
    <tr>
        <td colspan="2"> <small> source: <i> Visaniy, M.; Kieu, V.C.; Fornes, A.; Journet, N., "ICDAR 2013 Music Scores Competition: Staff Removal," Document Analysis and Recognition (ICDAR), 2013 12th International Conference on , vol., no., pp.1407,1411, 25-28 Aug. 2013</i> </small> </td>
    </tr>
</table>

In a nutshell, $W$-operators are pixel classifiers that use only a small neighborhood around each point to determine its output value. The shape of the neighborhood is called *window*. The *Feature Extraction* phase consists in sliding the *window* over all pixels in the training images and recording the pairs *pattern-label* observed, forming a *training set* to train a *classifier*. This process is repeated to transform an image, but with the classifier filling the output image with the predicted pixel values. 

### Install 

(aqui vou colocar as futuras instruções de instalações usando pip.)

Check if the *trios* module is installed by running

In [2]:
import trios

### Loading images and imagesets

The class `trios.Imageset` represents the sets of image pairs used to train $W$-operators. It is a subclass of `list` that stores tuples of 2 (`(input, output)`) or 3 (`(input, output, mask)`) elements. If a mask is present only the pixels which are nonzero at the mask are processed. 

`Imageset`s can be created in code as a list or we can use the methods `read`/`write` to load/save them from/to the disk.

In [3]:
imgset2 = trios.Imageset([('jung/input1.png', 'jung/output1.png')]) 
print(imgset2)
imgset = trios.Imageset.read('jung/level1.set')
print(imgset)

[('jung/input1.png', 'jung/output1.png')]


FileNotFoundError: [Errno 2] No such file or directory: 'jung/level1.set'

Use the `trios.shortcuts.persistence.load_image` function to load an image to use with `trioslib` and `trios.persistence.save_image` to save a result to the disk.

### Windows

Windows are just 2D numpy arrays of type `uint8` where a point belongs to the window if its value is not zero. We can also create windows using the `trios.shortcuts.window` module. Choosing a good window is very important, as the subsequent steps depend on the distribution of the raw data to determine correctly the output pixel values. 

If the window choosen is to small we will not be able to discriminate spatially large patterns, no matter how clever are the feature extractor and classifier used. On the other hand, if a very big window is used, we risk overfitting the data and having bad generalization. Although some feature extraction methods and classifiers may mitigate this issue, it is still wise to carefully select windows. 

In [5]:
# Using numpy.ones
import numpy as np
win_np = np.ones((3, 3), np.uint8)
# Or with the functions from trios.shortcuts.window
from trios.shortcuts import window
print('Square\n', window.square(5))
print('Rectangle\n', window.rectangle(9, 7))

Square
 [[1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]]
Rectangle
 [[1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]]


### Training W-operators

The `WOperator` class requires three parameters: a *window* (numpy array of type `np.uint`), a *FeatureExtractor* and a *Classifier*. Operators with very different characteristics can be obtained by combining these three components. In this tutorial we build a $W$-operator that uses pixel values as features (*RAWFeatureExtractor*) and a Decision Tree as classifier.

We also demonstrate an important feature of `trioslib`: any classification method present in [scikit-learn](http://scikit-learn.org/) can be used as the classifier in a $W$-operator by wrapping it in the `trios.classifier.SKClassifier`.

In [ ]:
from trios.feature_extractors import RAWFeatureExtractor
from trios.classifiers import SKClassifier

win = window.square(5)
wop = trios.WOperator(win, SKClassifier(DecisionTreeClassifier()), RAWFeatureExtractor)
wop.train(imageset)

### Applying W-operators and assessing their performance

apply in an image using wop.apply, get error rate using wop.eval. Comment about multiprocessing stuff.

### Saving and loading trained operators from disk

A operação da TRIOS está dividida em duas etapas: Extração de Características e Classificação. A primeira etapa é executada por objetos do tipo `FeatureExtractor` e, basicamente, posiciona a janela uma janela `window` sobre todos os pontos de todas as imagens de um `Imageset`. Em cada posição um `FeatureExtractor` faz um processamento local e extrai um vetor de características. Na segunda etapa um objeto do tipo `Classifier` é usado para determinar o valor de cada ponto da imagem de saída usando apenas o vetor de características extraído naquela posição na etapa anterior. Como toda informação usada é local, os operadores treinados são chamados de *window operators* ou $W-$operators.


A operação mais simples possível é simplesmente copiar os valores de cada ponto da janela em um vetor. Isto é feito pelas classes `RAWFeatureExtractor` e `RAWBitFeatureExtractor`. A classe `RAWBit...` funciona somente para imagens binárias e codifica as features nos bits de um `uint32`. Ja a classe `RAWFeatureExtractor` funciona para imagens de cinza também, mas consome mais memória. 

Abaixo um exemplo de criação e utilização de um `FeatureExtractor`. O método `extract_dataset` faz a extração de características de um conjunto de imagens. Se o segundo parâmetro for `True` esta função retorna uma matrix `X` e um vetor `y` tal que cada linha de `X` contem um padrão extraído da imagem e sua posição correspondente em `y` contem sua saída esperada. Os padrões são armazenados em `X` na ordem em que são encontrados nas imagens.

Se o segundo parâmetro for `False` o método retorna um dicionário que contém nas chaves os padrões encontrados e nos valores uma lista com a quantidade de vezes que um certo padrão apareceu em conjunto com cada valor de saída. É importante frisar que não é possível localizar a posição que um padrão ocorreu originalmente nas imagens de entrada.

In [ ]:
from trios.feature_extractors import RAWBitFeatureExtractor

fext = RAWBitFeatureExtractor(window)
dict_dataset = fext.extract_dataset(images, False)
X, y = fext.extract_dataset(images, True)
print(X.shape, X[100], y[100])

keys = list(dict_dataset.keys())
print(len(keys), dict_dataset[keys[10]])

A trios contém diversos classificadores no pacote `trios.classifiers`. Selecionamos o *ISI (Incremental Splitting of Intervals)* para trabalhar com imagens binárias. Este classificador requer que utilizemos um `RAWBitFeatureExtractor` como extrator de características.

In [ ]:
#from trios.classifiers import ISI
from trios.classifiers.ISI.isi import ISI
cls = ISI(window)

Para construímos um $W-$operador simplesmente passamos estes três objetos para o construtor da classe. Suas duas principais funções são `train`, que recebe um conjunto de imagens e executa o treinamento do operador, e `apply`, que transforma uma imagem usando o operador treinado. 

In [ ]:
wop = trios.WOperator(window, cls, fext)
d = wop.train(images[:-1])

In [ ]:
import scipy as sp
import scipy.ndimage
path = images[-1][0]
img = sp.ndimage.imread(path, mode='L')

res = wop.apply(img, img)

from matplotlib.pyplot import imshow
from matplotlib.cm import gray
import matplotlib.pyplot as plt
plt.subplot(121)
imshow(img, cmap=gray)
plt.subplot(122)
imshow(res, cmap=gray)

Operadores treinados pela TRIOS podem ser salvos usando o módulo `pickle` (opcionalmente em conjunto com o módulo `gzip`, para criar arquivos menores).

In [ ]:
import pickle
import gzip

with gzip.open('operador-salvo.gz', 'w') as f:
    pickle.dump(wop, f, -1)
    
with gzip.open('operador-salvo.gz') as f:
    wop2 = pickle.load(f)